In [17]:
import os
import cv2
import dlib
import numpy as np
from PIL import Image, ImageSequence
from google.colab import drive

In [10]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Set folder paths
input_folder = "/content/drive/MyDrive/Group Photos"  # Folder containing input group photos
output_folder = "/content/drive/MyDrive/extracted_group_faces"  # Folder to save extracted faces
os.makedirs(output_folder, exist_ok=True)

In [13]:
# Initialize the Dlib face detector
detector = dlib.get_frontal_face_detector()

In [14]:
# Function to check face clarity using Laplacian variance
def is_clear_face(face_img, threshold=100):
    gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_face, cv2.CV_64F).var()
    return laplacian_var > threshold, laplacian_var  # Return clarity status and variance

In [19]:
# Function to process each image frame
def process_frame(frame, file_name):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_frame)

    # Create a separate folder for this image's extracted faces
    image_name = os.path.splitext(file_name)[0]
    image_output_folder = os.path.join(output_folder, image_name)
    os.makedirs(image_output_folder, exist_ok=True)

    clear_faces_count = 0

    for i, face in enumerate(faces):
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())
        face_img = frame[y:y+h, x:x+w]

        is_clear, sharpness = is_clear_face(face_img)

        if is_clear:
            clear_faces_count += 1
            face_gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)  # Convert face to grayscale
            face_output_path = os.path.join(image_output_folder, f"face_{i+1}.jpg")
            cv2.imwrite(face_output_path, face_gray)
            print(f"Saved grayscale clear face {i+1} from {file_name} (Sharpness: {sharpness:.2f}) to {image_output_folder}")

    print(f"Image '{file_name}' has {clear_faces_count} clear face(s).")

    # Display results (optional)
    for face in faces:
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [20]:
# Process each image in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)

    try:
        if file_name.lower().endswith('.gif'):
            with Image.open(file_path) as img:
                for frame_number, frame in enumerate(ImageSequence.Iterator(img)):
                    frame = cv2.cvtColor(np.array(frame.convert('RGB')), cv2.COLOR_RGB2BGR)
                    process_frame(frame, file_name)

        else:
            original_img = cv2.imread(file_path)
            if original_img is None:
                print(f"Skipping file {file_name} as it is not a valid image.")
                continue

            process_frame(original_img, file_name)

    except Exception as e:
        print(f"Failed to process {file_name}: {e}")

Saved grayscale clear face 1 from A.jpg (Sharpness: 174.95) to /content/drive/MyDrive/extracted_group_faces/A
Saved grayscale clear face 2 from A.jpg (Sharpness: 209.66) to /content/drive/MyDrive/extracted_group_faces/A
Saved grayscale clear face 3 from A.jpg (Sharpness: 1708.45) to /content/drive/MyDrive/extracted_group_faces/A
Image 'A.jpg' has 3 clear face(s).
Saved grayscale clear face 1 from D.jpg (Sharpness: 391.66) to /content/drive/MyDrive/extracted_group_faces/D
Saved grayscale clear face 2 from D.jpg (Sharpness: 426.95) to /content/drive/MyDrive/extracted_group_faces/D
Image 'D.jpg' has 2 clear face(s).
Saved grayscale clear face 1 from B.jpg (Sharpness: 929.31) to /content/drive/MyDrive/extracted_group_faces/B
Saved grayscale clear face 2 from B.jpg (Sharpness: 123.34) to /content/drive/MyDrive/extracted_group_faces/B
Saved grayscale clear face 3 from B.jpg (Sharpness: 787.25) to /content/drive/MyDrive/extracted_group_faces/B
Saved grayscale clear face 4 from B.jpg (Sharpnes